# Derangements

A *derangement* is a permutation of $\{1, \dots, n\}$ that has no fixed points. We'd like to find the probability of a derangement if all permutations are equally likely.

The number of derangements $D_n$ is equal to the number of permutations minus the number of permutations that fix at least one point. Let $A_p$ denote the set of permutations that leave the $p$th item in its place, for $1 \leq p \leq n$. By the inclusion-exclusion principle, the number of permutations that leave at least one item in its place is

\begin{align*}
\lvert \cup_{p=1}^n A_p \rvert &= \sum_{p=1}^n \lvert A_p \rvert - \sum_{p < q} \lvert A_p \cap A_q \rvert + ... + (-1)^n \lvert \cap_{p=1}^n A_p \rvert
\end{align*}

Let $T \subseteq \{1, \dots, n\}$ be some set of points with $\lvert T \rvert = k$. If we fix the points of $T$, then we can freely permute the remaining $n-k$, so there are $(n-k)!$ such permutations that fix $T$. There are ${n \choose k}$ such subsets $T$ that have cardinality $k$. Therefore the $k$th term in the sum above is ${n \choose k}(n-k)!$. Plugging in, we get

\begin{align*}
\lvert \cup_{p=1}^n A_p \rvert &= {n \choose 1}(n-1)! - {n \choose 2}(n-2)! + \dots + (-1)^{n-1}{n \choose {n-1}} + (-1)^n \\
&= \sum_{k=1}^n (-1)^k{n \choose k}(n-k)!
\end{align*}

And thus, the number of derangements is

$$D_n = n! - \sum_{k=1}^n (-1)^k{n \choose k}(n-k)! = \sum_{k=0}^n (-1)^k{n \choose k}(n-k)!$$

The probability of any particular permutation is $1/n!$, so the probability of a derangement is

$$\frac{D_n}{n!} = \sum_{k=0}^n (-1)^k{n \choose k}\frac{(n-k)!}{n!} = \sum_{k=0}^n \frac{(-1)^k}{k!}$$

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import math
import numpy as np
import sys
sys.path.append('../modules')
from sample import permute_indices, fykd
from sha256prng import SHA256
from prng import lcgRandom
from scipy.misc import comb

In [2]:
def prob_derangement(n):
    fp_prob = np.ones(n+1)
    for k in range(1, len(fp_prob)):
        fp_prob[k] = -1/k
    fp_prob = np.cumprod(fp_prob)
    return sum(fp_prob)

def prob_derangement2(n):
    fp_prob = np.ones(n+1)
    for k in range(1, len(fp_prob)):
        fp_prob[k] = fp_prob[k-1] * (-1/k)
    return sum(fp_prob)

In [3]:
%timeit prob_derangement(100)
%timeit prob_derangement2(100)

10000 loops, best of 3: 30.9 µs per loop
10000 loops, best of 3: 44.2 µs per loop


In [4]:
print(1/np.exp(1), prob_derangement(100))

0.367879441171 0.367879441171


In [35]:
def check_derangement(vec, perm):
    '''
    Check whether perm is a derangement of vec
    Inputs must be numpy arrays
    '''
    
    return not any(np.equal(vec, perm))

def check_derangement2(vec, perm):
    
    anyequal = np.prod(vec-perm)
    return not bool(anyequal)

In [36]:
vec = np.array([1,2,3,4,5])
perm = np.array([2,3,4,5,1])

%timeit check_derangement(vec, perm)
%timeit check_derangement2(vec, perm)

The slowest run took 11.60 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 1.71 µs per loop
The slowest run took 9.71 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 3.07 µs per loop


# Permutation functions

We include two -- PIKK with k=n and Knuth shuffle.

# SPRT

In [40]:
def sequential_derangement_test(sampling_function, n, alpha, beta, multiplier, maxsteps=10**5):
    '''
    Conduct Wald's SPRT for whether derangements occur more or less frequently than 1/e
    H_0: derangements occur with equal frequency (p approx 1/e)
    H_1: p = p1 = multiplier * p0, multiplier in the range (0, 1/p0)
    
    sampling_function: a function which generates a random permutation
    n: number of items
    alpha: desired type 1 error rate
    beta: desired type 2 error rate
    multiplier: value in (0, 1/p0). Determines alternative hypothesis
    maxsteps: maximum number of trials before stopping the test. Default is 10**5.
    '''

    assert multiplier > 1
    assert maxsteps > 0

    # Set p0 = probability of a derangement
    p0 = prob_derangement(n)
    p1 = multiplier*p0
    assert p1 < 1
    assert p0 < 1
    
    # Set parameters
    lower = beta/(1-alpha)
    upper = (1-beta)/alpha
    lr_occurs = p1/p0
    lr_doesnotoccur = (1 - p1)/(1 - p0)

    LR = [1]
    decision = None        
    vec = np.array(range(0, n))
    steps = 0
    
    # Draw samples
    while lower < LR[-1] < upper and steps < maxsteps:
        steps += 1
        perm = sampling_function(n)
        Dn = check_derangement(vec, perm)

        if Dn:
            LR.append(LR[-1] * lr_occurs)
        else:
            LR.append(LR[-1] * lr_doesnotoccur)
            
        # Run test at step n
        if LR[-1] <= lower:
            # accept the null and stop
            decision = 0
            break

        if LR[-1] >= upper:
            # reject the null and stop
            decision = 1
            break
            
    return {'decision' : decision,
            'lower' : lower,
            'LR' : LR,
            'upper' : upper,
            'steps' : steps,
            'pvalue' : min(1/LR[-1], 1)
            }

In [50]:
alpha = 0.05
beta = 0
multiplier = 1.1

## RANDU

In [51]:
prng = lcgRandom(100) # from random.org Timestamp: 2017-01-14 22:56:40 UTC
sampling_func = lambda n: permute_indices(n, prng)
print(sampling_func(5))

print(sampling_func(5))

print(sampling_func(5))

res = sequential_derangement_test(sampling_func, n=100, alpha=alpha, beta=beta, multiplier=multiplier)
res['decision'], res['pvalue'], res['steps']

[0 1 2 4 3]
[3 2 0 1 4]
[3 0 1 4 2]


(None, 1, 100000)

In [52]:
prng = lcgRandom(100) # from random.org Timestamp: 2017-01-14 22:56:40 UTC
sampling_func = lambda n: fykd(np.array(range(n)), prng)
print(sampling_func(5))

print(sampling_func(5))

print(sampling_func(5))

res = sequential_derangement_test(sampling_func, n=100, alpha=alpha, beta=beta, multiplier=multiplier)
res['decision'], res['pvalue'], res['steps']

[0 1 2 3 4]
[1 0 3 2 4]
[2 0 1 3 4]


(None, 1, 100000)

## Super-Duper

In [53]:
A_SD = 0
B_SD = 69069
M_SD = 2**32
sdlcg = lcgRandom(seed=547691802, A=A_SD, B=B_SD, M=M_SD) 
sampling_func = lambda n: permute_indices(n, sdlcg)

res = sequential_derangement_test(sampling_func, n=100, alpha=alpha, beta=beta, multiplier=multiplier)
res['decision'], res['pvalue'], res['steps']

(None, 1, 100000)

In [54]:
A_SD = 0
B_SD = 69069
M_SD = 2**32
sdlcg = lcgRandom(seed=547691802, A=A_SD, B=B_SD, M=M_SD) 
sampling_func = lambda n: fykd(np.array(range(n)), sdlcg)

res = sequential_derangement_test(sampling_func, n=100, alpha=alpha, beta=beta, multiplier=multiplier)
res['decision'], res['pvalue'], res['steps']

(None, 1, 100000)

## Mersenne Twister

In [55]:
prng = np.random
prng.seed(547691802)
sampling_func = lambda n: permute_indices(n, prng)

res = sequential_derangement_test(sampling_func, n=100, alpha=alpha, beta=beta, multiplier=multiplier)
res['decision'], res['pvalue'], res['steps']

(None, 1, 100000)

In [56]:
prng = np.random
prng.seed(547691802)
sampling_func = lambda n: fykd(np.array(range(n)), prng)

res = sequential_derangement_test(sampling_func, n=100, alpha=alpha, beta=beta, multiplier=multiplier)
res['decision'], res['pvalue'], res['steps']

(None, 1, 100000)

## SHA-256

In [57]:
prng = SHA256(547691802)
sampling_func = lambda n: permute_indices(n, prng)

res = sequential_derangement_test(sampling_func, n=100, alpha=alpha, beta=beta, multiplier=multiplier)
res['decision'], res['pvalue'], res['steps']

(1, 0.047783798217551771, 317)

In [58]:
prng = SHA256(547691802)
sampling_func = lambda n: fykd(np.array(range(n)), prng)

res = sequential_derangement_test(sampling_func, n=100, alpha=alpha, beta=beta, multiplier=multiplier)
res['decision'], res['pvalue'], res['steps']

(None, 1, 100000)